In [ ]:
from pathlib import Path
import re
import subprocess
import urllib.parse

import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
# Create the _data directory and its subdirectories, if they don't exist
data_dir = Path('../_data')
subdirs = ['processed','raw']

for subdir in subdirs:
    folder_path = data_dir / subdir
    folder_path.mkdir(parents=True, exist_ok=True)
processed_folder = (data_dir / 'processed')

In [ ]:
def convertToPMTileDirectly(path, processed_folder):
    """
    Geotiff values need to be between 0 and 255; Values will be converted into Ints. 

    Either use ArcPro/QGIS's raster calc to norm values 0-255 OR `rio calc "(* 2.0 (read 1))" input.tif test.tif`
    
    Use the more advanced func to get floats and other numbers. 
    """
    if path.exists():
        # convert to mbtiles
        from_path = path.absolute()
        to_path = processed_folder / (path.stem +'.mbtiles')
        # use j to limit the amount of cores so it does not use too much memory (GDAL_CACHEMAX might also fix the issue??)
        cmd = ['rio', 'mbtiles', str(from_path), str(to_path),'--overwrite','-#','-j','6','--co','ZLEVEL=3','--format','PNG','--zoom-levels','10..16','--tile-size','1024']
        p = subprocess.Popen(cmd, stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line)
        p.wait()

        # convert to pmtiles
        from_path = processed_folder / (path.stem +'.mbtiles')
        to_path = processed_folder / (path.stem +'.pmtiles')
        cmd = ['pmtiles','convert',str(from_path), str(to_path)]
        p = subprocess.Popen(cmd, stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line)
        p.wait()
        return to_path
    else:
        print('file does not exist')
        return 

In [ ]:
path = Path('../_data/raw/ST_Clipped_20190922.tif')
def get_all_bands_range(file_path):
    with rasterio.open(file_path) as src:
        ranges = []
        for band in range(1, src.count + 1):
            data = src.read(band)
            if src.nodata is not None:
                valid_data = data[data != src.nodata]
            else:
                valid_data = data
            ranges.append((np.nanmin(valid_data), np.nanmax(valid_data)))
        return ranges

path = Path('../_data/raw/ST_Clipped_20190922.tif')
ranges = get_all_bands_range(path)

for i, (min_val, max_val) in enumerate(ranges, 1):
    print(f"Band {i} range: {min_val} to {max_val}")

If you get `Failed to convert .., Failed to create SQL statement, SQL logic error: no such table: metadata\n'` run the function again.

In [ ]:
convertToPMTileDirectly(path, processed_folder)

### Testing 

https://github.com/tilezen/joerd/blob/master/docs/formats.md#terrarium

-32,768 to 32,767 meters with centimeter precision. (8 decimal places) 

In [ ]:
import subprocess
from pathlib import Path
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

def encode_terrarium(elevation):
    """
    Encode elevation data into Terrarium PNG format.
    Input: elevation in meters
    Output: RGB values (0-255) for PNG encoding
    """
    # Handle NaN values
    elevation = np.nan_to_num(elevation, nan=-32768)
    
    # Add offset to handle negative values
    v = elevation + 32768
    
    # Calculate RGB channels
    r = np.floor(v / 256).astype(np.uint8)
    g = np.floor(v % 256).astype(np.uint8)
    b = np.floor((v - np.floor(v)) * 256).astype(np.uint8)
    
    # Ensure values are within valid range
    r = np.clip(r, 0, 255)
    g = np.clip(g, 0, 255)
    b = np.clip(b, 0, 255)
    
    return np.stack([r, g, b], axis=0)

def convertToPMTileTerrarium(path, processed_folder):
    """
    Convert elevation GeoTIFF to PMTiles using Terrarium encoding.
    Handles float values and negative elevations.
    """
    if not path.exists():
        print('File does not exist')
        return
    
    # Create temporary file for the encoded raster
    temp_tiff = processed_folder / f"{path.stem}_encoded.tif"
    
    with rasterio.open(path) as src:
        # Read the elevation data
        elevation = src.read(1)
        
        # Encode to Terrarium format
        rgb = encode_terrarium(elevation)
        
        # Create output raster with same georeferencing but 3 bands
        kwargs = src.meta.copy()
        kwargs.update({
            'count': 3,
            'dtype': 'uint8',
            'driver': 'GTiff',
            'compress': 'lzw',
            'nodata': None,
            'tiled': True,
            'blockxsize': 256,
            'blockysize': 256
        })
        
        # If not in Web Mercator, reproject
        if src.crs != 'EPSG:3857':
            transform, width, height = calculate_default_transform(
                src.crs, 'EPSG:3857', src.width, src.height, *src.bounds)
            kwargs.update({
                'crs': 'EPSG:3857',
                'transform': transform,
                'width': width,
                'height': height
            })
            
            # Initialize reprojected RGB array
            rgb_reprojected = np.zeros((3, height, width), dtype=np.uint8)
            
            for i in range(3):
                reproject(
                    source=rgb[i],
                    destination=rgb_reprojected[i],
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs='EPSG:3857',
                    resampling=Resampling.bilinear
                )
            rgb = rgb_reprojected
        
        # Write encoded raster
        with rasterio.open(temp_tiff, 'w', **kwargs) as dst:
            dst.write(rgb)
    
    # Convert to mbtiles
    mbtiles_path = processed_folder / f"{path.stem}.mbtiles"
    if mbtiles_path.exists():
        mbtiles_path.unlink()
        
    cmd = [
        'rio', 'mbtiles',
        str(temp_tiff),
        str(mbtiles_path),
        '--title', path.stem,
        '--description', 'Elevation data in Terrarium encoding',
        '-j', '6',
        '--co', 'ZLEVEL=3',
        '--format', 'PNG',
        '--zoom-levels', '10..16',
        '--tile-size', '1024'
    ]
    
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line)
    p.wait()
    
    # Convert to pmtiles
    pmtiles_path = processed_folder / f"{path.stem}.pmtiles"
    cmd = ['pmtiles', 'convert', str(mbtiles_path), str(pmtiles_path)]
    
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line)
    p.wait()
    
    # Clean up temporary files
    if temp_tiff.exists():
        temp_tiff.unlink()
    # if mbtiles_path.exists():
    #     mbtiles_path.unlink()
    
    return pmtiles_path

In [ ]:
convertToPMTileTerrarium(Path('../_data/raw/ST_Clipped_20190922.tif'), processed_folder)